***ELMo 표현을 사용한 스팸메일 분류***

***tensorflow hub로부터 학습된 ELMo를 받아오기 위한 패키지 설치***

In [2]:
!pip3 install tensorflow-hub
!git clone https://github.com/stedy/Machine-Learning-with-R-datasets.git

fatal: destination path 'Machine-Learning-with-R-datasets' already exists and is not an empty directory.


In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K

sess = tf.Session()
K.set_session(sess)
# 세션 초기화. 이는 텐서플로우 개념.

elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

Using TensorFlow backend.


In [4]:
# csv 파일 처리용 패키지 사용
import pandas as pd
data = pd.read_csv('Machine-Learning-with-R-datasets/sms_spam.csv', encoding='latin-1')
data[:5]

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
# csv 데이터 label, sentence 구분
data['type'] = data['type'].replace(['ham','spam'],[0,1])
y_data = list(data['type'])
X_data = list(data['text'])

In [6]:
X_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [7]:
print(y_data[:5])

[0, 0, 1, 0, 0]


In [9]:
# 학습, 테스트 데이터 80%, 20%로 구분.
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)          
print(n_of_test)

5574
4459
1115


In [0]:
import numpy as np
X_train = np.asarray(X_data[:n_of_train]) 
y_train = np.asarray(y_data[:n_of_train]) 
X_test = np.asarray(X_data[n_of_train:]) 
y_test = np.asarray(y_data[n_of_train:])

In [0]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

In [12]:
# 
from keras.models import Model
from keras.layers import Dense, Lambda, Input

# keras ELMO + dense, sigmoid, binary classification 모델 선언
input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0802 05:33:03.029516 139964920350592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 05:33:03.032299 139964920350592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 05:33:03.422116 139964920350592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 05:33:03.451229 139964920350592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0802 05:33:03.472887 139964920350592 deprecation_wrappe

In [13]:
# 학습 진행
history = model.fit(X_train, y_train, epochs=1, batch_size=60)

W0802 05:33:17.556728 139964920350592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
4459/4459 [==============================] - 1161s 260ms/step - loss: 0.1334 - acc: 0.9493


In [14]:
# 모델 성능 비교.
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

1115/1115 [==============================] - 279s 250ms/step

 테스트 정확도: 0.9803
